In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Normalization, Dense, InputLayer
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError,Huber
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
data=pd.read_csv('train.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
sns.pairplot(data[['years','km','rating','condition','economy','top speed','hp','torque','current price']])

In [ ]:
tensor_data=tf.constant(data)
tensor_data=tf.cast(tensor_data,tf.float32)
tensor_data=tf.random.shuffle(tensor_data)
print(tensor_data)

In [ ]:
tensor_data.shape

In [ ]:
X=tensor_data[:,3:-1]
y=tensor_data[:,-1]
y=tf.expand_dims(y,axis=-1)

In [ ]:
X[0][-1]

In [ ]:
y[0]

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
TRAIN_RETIO=0.8
VAL_RETIO=0.1
TEST_RETIO=0.1
DATASET_SIZE=len(X)
DATASET_SIZE

In [ ]:
X_train=X[:int(TRAIN_RETIO*DATASET_SIZE)]
y_train=y[:int(TRAIN_RETIO*DATASET_SIZE)]
X_val=X[int(TRAIN_RETIO*DATASET_SIZE):int((TRAIN_RETIO+VAL_RETIO)*DATASET_SIZE)]
y_val=y[int(TRAIN_RETIO*DATASET_SIZE):int((TRAIN_RETIO+VAL_RETIO)*DATASET_SIZE)]
X_test=X[int((TRAIN_RETIO+VAL_RETIO)*DATASET_SIZE):]
y_test=y[int((TRAIN_RETIO+VAL_RETIO)*DATASET_SIZE):]
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((X_train,y_train))
train_dataset=train_dataset.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset=tf.data.Dataset.from_tensor_slices((X_val,y_val))
val_dataset=val_dataset.shuffle(buffer_size=8,reshuffle_each_iteration=True).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
len(train_dataset),len(val_dataset)

In [ ]:
normalize=Normalization()
normalize.adapt(X_train)
normalize(X)[1]

In [ ]:
model=tf.keras.Sequential([
    InputLayer(input_shape=(8,)),
    normalize,
    Dense(128,activation='relu'),
    Dense(128,activation='relu'),
    Dense(128,activation='relu'),
    Dense(1)])
tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.1),loss=MeanAbsoluteError(),metrics=RootMeanSquaredError())


In [ ]:
model.summary()

In [ ]:
history=model.fit(train_dataset,validation_data=val_dataset,epochs=100,verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['root_mean_squared_error'])
plt.plot(history.history['val_root_mean_squared_error'])
plt.title('model performance')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
model.predict(tf.expand_dims(X_test[0], axis = 0 ))

In [ ]:
y_test[0]

In [ ]:
y_true = list(y_test[:,0].numpy())

In [ ]:
y_pred = list(model.predict(X_test)[:,0])
print(y_pred)

In [ ]:
ind = np.arange(100)
plt.figure(figsize=(40,20))

width = 0.1

plt.bar(ind, y_pred, width, label='Predicted Car Price')
plt.bar(ind + width, y_true, width, label='Actual Car Price')

plt.xlabel('Actual vs Predicted Prices')
plt.ylabel('Car Price Prices')

plt.show()